# Import Packages

In [1]:
import sys
import pandas as pd
import numpy as np


import re
import os
import xlsxwriter
import sklearn
import tweepy
import os.path  # downloading and uploading files, changing directories
import openpyxl
import ssl

# cleaning
import json
import requests
import emoji
import regex
import string
from collections import Counter

# visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

# NLP libraries
import spacy
import pyLDAvis
import gensim
import pyLDAvis.gensim_models
from spacy.tokenizer import Tokenizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
from bertopic import BERTopic
from nltk.book import *
import nltk # natural language processing e.g., tokenization
import tensorflow
import torch
torch.has_mps

# Linkage functions
from scipy.cluster import hierarchy as sch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bertopic/_utils.py:4: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/image_utils.py:222: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  def resize(self, image, size, resample=PIL.Image.BILINEAR, default_to_square=True, max_size=None):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/plotly/graph_objs/__init__

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


# Conversion csvs to dataframe 2019, 2020-01, 2020-02, 2020-03, 2020-04, 2020-05, 2020-06

In [2]:
# df = pd.read_csv ('file_name.csv')
# df = pd.read_csv(filename, header=None, names=cols)
#if (df.iloc[0] == cols).all():
    #df = df[1:].reset_index(drop=True)
cols = ["tweet_id", "BLM", "ALM", "Blue"]
# 2019
# 2020-01
df2020_01 = pd.read_csv("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-01 copy.csv") #inserted column names
# 2020-02
df2020_02 = pd.read_csv("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-02.csv", header=None, names=cols) #add column headers
if (df2020_02.iloc[0]==cols).all():
    df2020_02 = df2020_02[1:].reset_index(drop=True)
# 2020-03
# 2020-04
# 2020-05
# 2020-06

In [3]:
# check info, shape, and describe
df2020_01.info()
df2020_01.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100796 entries, 0 to 100795
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   tweet_id  100796 non-null  int64
 1   BLM       100796 non-null  int64
 2   ALM       100796 non-null  int64
 3   Blue      100796 non-null  int64
dtypes: int64(4)
memory usage: 3.1 MB


,tweet_id,BLM,ALM,Blue
0,1212161788533342208,0,0,1
1,1212161860600041475,1,0,0
2,1212162108596588544,1,0,0
3,1212162110727376897,1,0,0
4,1212162171544784899,1,0,0


In [4]:
df2020_02.info()
df2020_02.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280274 entries, 0 to 280273
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   tweet_id  280274 non-null  int64
 1   BLM       280274 non-null  int64
 2   ALM       280274 non-null  int64
 3   Blue      280274 non-null  int64
dtypes: int64(4)
memory usage: 8.6 MB


,tweet_id,BLM,ALM,Blue
0,1223396055263449088,0,0,1
1,1223396189451759621,1,0,0
2,1223396291813744641,0,1,0
3,1223396392602939402,1,0,0
4,1223396445266546688,1,0,0


# Remove ids that do not = BLM 1; for 2019 tweets only keep those from sept to december

In [5]:
blm_2020_01 = df2020_01.loc[df2020_01['BLM']==1]
blm_2020_02 = df2020_02.loc[df2020_02['BLM']==1]

In [6]:
# check shape and info
blm_2020_01.info()
blm_2020_01.describe()
blm_2020_02.info()
blm_2020_02.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40872 entries, 1 to 100795
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   tweet_id  40872 non-null  int64
 1   BLM       40872 non-null  int64
 2   ALM       40872 non-null  int64
 3   Blue      40872 non-null  int64
dtypes: int64(4)
memory usage: 1.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 188764 entries, 1 to 280272
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   tweet_id  188764 non-null  int64
 1   BLM       188764 non-null  int64
 2   ALM       188764 non-null  int64
 3   Blue      188764 non-null  int64
dtypes: int64(4)
memory usage: 7.2 MB


,tweet_id,BLM,ALM,Blue
count,1.887640e+05,188764.0,188764.000000,188764.000000
mean,1.226693e+18,1.0,0.008678,0.056790
std,2.275029e+15,0.0,0.092748,0.231442
min,1.223396e+18,1.0,0.000000,0.000000
25%,1.225509e+18,1.0,0.000000,0.000000
50%,1.225777e+18,1.0,0.000000,0.000000
75%,1.226633e+18,1.0,0.000000,0.000000
max,1.233905e+18,1.0,1.000000,1.000000


In [7]:
# check head

# Convert json files to dfs

In [18]:
pd.options.display.max_colwidth = 2000
import flatten_json
from flatten_json import flatten

## Flatten tweets

In [10]:
def read_json_file(file): #read in as json file
    with open(file, "r") as r:
        response = r.read()
        response = response.replace('\n', '')
        response = response.replace('}{', '},{')
        response = "[" + response + "]"
        return json.loads(response)

In [ ]:
data_2019 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2019/2019blm.jsonl")

In [ ]:
data_2020_01 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-01blm.jsonl")

In [ ]:
data_2020_02 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-02blm.jsonl")

In [ ]:
data_2020_03 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-03blm.jsonl")

In [ ]:
data_2020_04 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-04blm.jsonl")
#data_2020_05 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-05blm.jsonl")
#data_2020_06 = read_json_file("/Users/joni-leighwebster/Documents/GitHub/blm_twitter_corpus/twitter/2020/2020-06blm.jsonl")

In [19]:
tweets_2019 = pd.json_normalize((data_2019)) #flatten tweets using flatten_json library
tweets_2020_01 = pd.json_normalize(data_2020_01)
tweets_2020_02 = pd.json_normalize(data_2020_02)
tweets_2020_03 = pd.json_normalize(data_2020_03)
tweets_2020_04 = pd.json_normalize(data_2020_04)
#tweets_2020_05 = pd.json_normalize(data_2020_05)
#tweets_2020_06 = pd.json_normalize(data_2020_06)

In [13]:
## select only certain columns (learn how to create a function for this)
flat_2019 = tweets_2019[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

flat_2020_01 = tweets_2020_01[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

flat_2020_02 = tweets_2020_02[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

flat_2020_03 = tweets_2020_03[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

flat_2020_04 = tweets_2020_04[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

#flat_2020_05 = tweets_2020_05[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

#flat_2020_06 = tweets_2020_06[['created_at', 'id_str', 'full_text', 'place', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted',  'lang', 'entities.hashtags', 'entities.symbols', 'user.id_str', 'user.location', 'user.followers_count', 'user.friends_count', 'in_reply_to_status_id_str', 'retweeted_status.full_text', 'quoted_status_id_str', 'user.statuses_count']]

## Select for language

In [23]:
# check tweets to see if random sample is presented in engilsh

In [15]:
# keep if language is in en (create a function for this)
#en_2020_01 = flat_2020_01.loc[flat_2020_01['lang']=='en']

In [16]:
# check info
eng_2020_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59322 entries, 0 to 65824
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   tweet_id_str           59322 non-null  int64              
 1   tweet_id               59322 non-null  int64              
 2   created_at             59322 non-null  datetime64[ns, UTC]
 3   full_text              59322 non-null  object             
 4   entities               59322 non-null  object             
 5   in_reply_to_status_id  2875 non-null   float64            
 6   in_reply_to_user_id    3378 non-null   float64            
 7   user                   59322 non-null  object             
 8   retweeted_status       40089 non-null  object             
 9   is_quote_status        59322 non-null  bool               
 10  retweet_count          59322 non-null  int64              
 11  favorite_count         59322 non-null  int64          

# Merge csv file based on tweet id on left and remove rows with no tweet information

# Export tweets for analysis - list of user_ids

## a. random sample of tweet_ids (.333 of each  df by date)

## b. keep only the user_ids

# Save as new file by year and reupload data for analysis

In [ ]:
# combine all datasets into new dataframe

# Descriptive statistics on those ids available

## a. Summary data

## b. Graphic representations (trend graphs of important data # of tweets, # of tweets by demographic characteristics victim or tweeter)